In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from lime import lime_image
from skimage.segmentation import mark_boundaries, slic
from skimage.color import gray2rgb, rgb2gray


In [ ]:
# Load model
model = tf.keras.models.load_model('CNNFS.h5')

In [ ]:
import os

image_path = r"C:\Users\Administrator\CNN Code - Github\Data\company_11508_image.png"
if not os.path.exists(image_path):
    print("The file does not exist at the specified path.")
else:
    print("File found at the specified path.")


In [ ]:
# Load and preprocess the image
#img_path = r'C:\Users\Administrator\FS36\FS0\NonBankruptcy\company_13311_image.png'  
#img = image.load_img(img_path, target_size=(224, 224))  # Resize to match your model's input
#img_array = image.img_to_array(img)
#img_array = np.expand_dims(img_array, axis=0)
#img_array /= 255.

# Load and preprocess the image
image_path = r"C:\Users\Administrator\CNN Code - Github\Data\Bankruptcy\company_11508_image.png"
img = cv2.imread(image_path)
img = cv2.resize(img, (224, 224))
img = np.reshape(img, [1, 224, 224, 3])

In [ ]:
# Define the prediction function
def predict_fn(images):
    predictions = model.predict(images)
    predicted_probability = predictions[0][0]
    predicted_class = int(predicted_probability > 0.5)
    return predicted_class, predicted_probability

In [ ]:
# Call the prediction function and capture the returned values
predicted_class, predicted_probability = predict_fn(img)

In [ ]:
# Define class labels
class_labels = {0: 'NonBankrupt', 1: 'Bankrupt'}

In [ ]:
print("Predicted class:", predicted_class)
print("Predicted probability:", predicted_probability)

In [ ]:
# Make prediction to obtain predicted class and its probability
predicted_class, predicted_probability = predict_fn(img)
predicted_class_label = class_labels[predicted_class]

In [ ]:
# Generate superpixels for visualization
superpixels = slic(img[0], n_segments=100, compactness=10, sigma=1)

In [ ]:
# Create a LimeImageExplainer
explainer = lime_image.LimeImageExplainer()

In [ ]:
# Wrap the original model prediction function for LIME
def lime_predict_fn(images):
    # Ensure images are batched correctly
    batched_images = np.stack(images, axis=0)
    predictions = model.predict(batched_images)
    return predictions

In [ ]:
# Explain the model's prediction using LIME
explanation = explainer.explain_instance(
    img[0], 
    lime_predict_fn,  # Use the wrapped prediction function for LIME
    top_labels=1, 
    hide_color=0, 
    num_samples=200
)

In [ ]:
# Get the weights for each superpixel
weights = explanation.local_exp[explanation.top_labels[0]]
weighted_superpixels = np.zeros(img[0].shape[:2])

In [ ]:
# Prepare the mask
for weight, superpixel in weights:
    weighted_superpixels[explanation.segments == superpixel] = weight

In [ ]:
# Visualize the segmented image with superpixels
plt.imshow(mark_boundaries(img[0], superpixels))
plt.title(f'Segmented Image - Predicted Class: {predicted_class_label}')
plt.show()

In [ ]:
# Display the original image with positive superpixels highlighted
lime_img, mask = explanation.get_image_and_mask(
    explanation.top_labels[0], 
    positive_only=True, 
    num_features=5, 
    hide_rest=False
)
plt.imshow(mark_boundaries(lime_img, mask))
plt.title(f'LIME Explanation (Positive Contributions) - Predicted Class: {predicted_class_label}')
plt.show()

In [ ]:
# Optionally, display with both positive and negative superpixels
lime_img, mask = explanation.get_image_and_mask(
    explanation.top_labels[0], 
    positive_only=False, 
    num_features=10, 
    hide_rest=False
)
plt.imshow(mark_boundaries(lime_img, mask))
plt.title('LIME Explanation (Positive and Negative Contributions)')
plt.show()

In [ ]:
# Get the image array in RGB to overlay the colors
original_image = gray2rgb(rgb2gray(img[0]))  # Convert to grayscale first if it's colored

In [ ]:
# Retrieve the weights for the top class
top_label = explanation.top_labels[0]
weights = dict(explanation.local_exp[top_label])
segments = explanation.segments

In [ ]:
# Create a mask that will overlay the colors on the superpixels
mask = np.zeros(segments.shape)

for superpixel, weight in weights.items():
    if weight > 0:
        # Positive weights contribute to the prediction of the class (e.g., bankruptcy)
        mask[segments == superpixel] = weight
    else:
        # Negative weights detract from the prediction of the class
        mask[segments == superpixel] = weight

In [ ]:
# Apply the color scheme with a blue-red colormap (negative-positive)
plt.imshow(original_image)
plt.imshow(mask, cmap='RdBu', alpha=0.5)  # alpha for transparency
plt.colorbar()
plt.title(f'Most Influential Features for the Model - Predicted Class: {predicted_class_label}')
plt.show()